<a href="https://colab.research.google.com/github/RakinAli/AI-course/blob/main/Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preprocess
This code preprocesses the NASA TIF files and makes them ready for comparision




## Clipping CHM 10x10, CHM 1x1 and Dynamic World
This part of the code is reponsible for "clipping" Dynamic world, Canopy Height models so that for each TIF file from NASA, there is a corresponding CHM models for comparision and Dynamic World to filter out everything that is not a tree



In [ ]:
!pip install rasterio
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
"""
This code is reponsible for "clipping" Dynamic world, Canopy Height models so that for each TIF file from NASA, there is a corresponding CHM models for comparision and Dynamic World to filter out everything that is not a tree
"""
# Imports
import rasterio
import os
import ee
import time
import sys
from tqdm import tqdm
from datetime import datetime, timedelta
import pyproj
from pyproj import Transformer
import geemap
import geopandas as gpd
from shapely.geometry import box
from pathlib import Path
import requests
import shutil



# Directory in Google drive <-- Modify so that it points to your datasets
#DATASET_PATH = "/content/drive/MyDrive/Summer internship - Research/Canopy_height_summer2024/Dataset/"

# Earth Engine stuff
ee.Authenticate()
ee.Initialize(project="ee-rakinali00") #<-- change this

# Global Variables
CHM_10X10 = ee.Image("users/nlang/ETH_GlobalCanopyHeight_2020_10m_v1")
dynamic_world = ee.ImageCollection("GOOGLE/DYNAMICWORLD/V1")
CHM_1x1 = ee.ImageCollection("projects/meta-forest-monitoring-okw37/assets/CanopyHeight")
from pyproj import Transformer


ROOT_PATH = Path("/content/drive/MyDrive/Summer internship - Research/Canopy_height_summer2024/Dataset/2017/NASA TIF Files")
fl = 'ME_20170817_CMSX2_l0s99_CHM.tif'
with rasterio.open(ROOT_PATH/fl) as img:
    xmin, ymin, xmax, ymax = img.bounds
    transformer = Transformer.from_crs(img.crs, "EPSG:4326", always_xy=True)
    x1, y1 = transformer.transform(xmin,ymax)
    x2, y2 = transformer.transform(xmax,ymin)
aoi = ee.Geometry.Polygon([[x1, y1], [x1, y2], [x2, y2], [x2, y1], [x1, y1]])

START_DATE = ee.Date(str(2021)+ '-01-01')
END_DATE = START_DATE.advance(11,"month")
image = ee.ImageCollection("GOOGLE/DYNAMICWORLD/V1").filterDate(START_DATE, END_DATE).median().clip(aoi)
proj = ee.Projection('EPSG:32619')
SCALE = 10
if len(image.bandNames().getInfo())>0:
    url2 = image.getDownloadURL({'scale': SCALE,'region':aoi, 'format': "GEO_TIFF", 'formatOptions': {'cloudOptimized': True}, 'crs': proj})

    # Handle downloading the actual pixels.
    r = requests.get(url2, stream=True)
    if r.status_code != 200:
        r.raise_for_status()
    filename = 'down_test.tif'
    with open(filename, 'wb') as out_file:
        shutil.copyfileobj(r.raw, out_file)


In [ ]:
"""
This code is reponsible for "clipping" Dynamic world, Canopy Height models so that for each TIF file from NASA, there is a corresponding CHM models for comparision and Dynamic World to filter out everything that is not a tree
"""
# Imports
import rasterio
import os
import ee
import time
import sys
from tqdm import tqdm
from datetime import datetime, timedelta
import pyproj
from pyproj import Transformer
import geemap
import geopandas as gpd
from shapely.geometry import box
import requests
import shutil

# Directory in Google drive <-- Modify so that it points to your datasets
DATASET_PATH = "/content/drive/MyDrive/Summer internship - Research/Canopy_height_summer2024/Dataset/"

# Earth Engine stuff <-- MODIFY
ee.Authenticate()
ee.Initialize(project="ee-rakinali00") #<-- change this

# Global Variables
CHM_10X10 = ee.Image("users/nlang/ETH_GlobalCanopyHeight_2020_10m_v1")
dynamic_world = ee.ImageCollection("GOOGLE/DYNAMICWORLD/V1")
CHM_1x1 = ee.ImageCollection("projects/meta-forest-monitoring-okw37/assets/CanopyHeight")

# Check if paths exists. Returns true or sys.exit()
def check_path(path):
  if not os.path.exists(path):
    print("Path: ", path)
    print("Path does not exist")
    sys.exit()
  else:
    return True

def make_dir(path):
  if not os.path.exists(path):
    os.makedirs(path)


def get_dateRange(file):
    """
    The fileName contains the date. The date however is a bit hard to extract.
    The date is needed for Dynamic World. We try to extract the date from the
    FileName. This code was admittedly hardcoded
    """
    # Print the name of the file
    parts = file.split("_")

    # Determine which part of the filename contains the date
    date_str = None
    for part in parts:
        # Check if the part is in the YYYYMMDD format
        if len(part) == 8 and part.isdigit():
            date_str = part
            break
        # Check if the part has both digits and alphabets (e.g., 12March2017)
        elif any(char.isdigit() for char in part) and any(char.isalpha() for char in part):
            date_str = part
            break

    if date_str is None:
        print("File:", file)
        sys.exit("Cannot handle for now")

    try:
        # Check if the date_str is in the format YYYYMMDD
        if len(date_str) == 8 and date_str.isdigit():
            year = int(date_str[:4])
            month = int(date_str[4:6])
            day = int(date_str[6:])
            start_date = datetime(year, month, day)
        # Check if it has any alphabets, then it's Date-Name of Month-Year format
        elif any(char.isalpha() for char in date_str):
            try:
                start_date = datetime.strptime(date_str, "%d%B%Y")
            except ValueError:
                start_date = datetime.strptime(date_str, "%d%b%Y")
        else:
            print("File:", file)
            sys.exit("Cannot handle for now")

        end_date = start_date + timedelta(days=10)
    except ValueError as e:
        print(f"Error parsing date from file {file}: {e}")
        sys.exit("Cannot handle for now")

    return ee.Date(start_date.strftime("%Y-%m-%d")), ee.Date(end_date.strftime("%Y-%m-%d"))



def download_image(image,out_outdirectory,scale,bbox,crs,fileName):
    url = image.getDownloadURL({'scale': scale,'region':bbox, 'format': "GEO_TIFF", 'formatOptions': {'cloudOptimized': True}, 'crs': crs})
    r = requests.get(url, stream=True)
    if r.status_code != 200:
        r.raise_for_status()

    filename = os.path.join(out_outdirectory,fileName)
    with open(filename, 'wb') as out_file:
        shutil.copyfileobj(r.raw, out_file)

def get_bbox(raster_file_path, target_crs="EPSG:4326"):
    """
    Extracts the bounding box from a raster file, transforms it to the specified CRS,
    and returns the bounding box as an Earth Engine Geometry Polygon.

    Parameters:
    raster_file_path (str): Path to the raster file.
    target_crs (str): The target CRS to transform the bounding box coordinates. Default is "EPSG:4326".

    Returns:
    ee.Geometry.Polygon: The bounding box as an Earth Engine Geometry Polygon.
    """
    with rasterio.open(raster_file_path) as src:
        xmin, ymin, xmax, ymax = src.bounds
        transformer = Transformer.from_crs(src.crs, target_crs, always_xy=True)
        x1, y1 = transformer.transform(xmin, ymax)
        x2, y2 = transformer.transform(xmax, ymin)
        projection = src.crs.to_string()

    bbox = ee.Geometry.Polygon([[x1, y1], [x1, y2], [x2, y2], [x2, y1], [x1, y1]])
    return bbox, projection


def clipping(year):
    """
    Algorithm:
    1.  First check if path exists and there exists TIF files

    2.  For each TIF file, get the bounding box and clip the same area in CHM
        models and Dynamic world

    3.
    """
    tif_files_path = os.path.join(DATASET_PATH, year + "/NASA TIF Files/")
    if check_path(tif_files_path):
        total_files = len(os.listdir(tif_files_path))
        if total_files == 0:
            sys.exit("There are no files in NASA TIF Files")
        print("Total files:", total_files)

    for file in tqdm(os.listdir(tif_files_path)):
        bbox, projection = get_bbox(os.path.join(tif_files_path, file))

        # Start and end date of the file
        start_date, end_date = get_dateRange(file)
        dynamic_world_filter = (dynamic_world.filterDate(start_date, end_date).filterBounds(bbox)).median().clip(bbox)
        chm_1x1_filter = (CHM_1x1).filterBounds(bbox).first()

        # Output directory
        CHM_10X10_OUTDIR = os.path.join(DATASET_PATH, year + "/CHM_10X10")
        CHM_1x1_OUTDIR = os.path.join(DATASET_PATH, year + "/CHM_1x1/")
        dynamic_world_OUTDIR = os.path.join(DATASET_PATH, year + "/Dynamic_World/")
        make_dir(CHM_10X10_OUTDIR)
        make_dir(CHM_1x1_OUTDIR)
        make_dir(dynamic_world_OUTDIR)
        print("one more print below")
        if len(dynamic_world_filter.bandNames().getInfo()) > 0:
            fileName_dw = file[:-4] + "_dw.tif"
            download_image(dynamic_world_filter,dynamic_world_OUTDIR,10,bbox,projection,fileName_dw)
            print("I am working")

        else:
          print("File ",file ," has been skipped")
          continue
        download_image(CHM_10X10,CHM_10X10_OUTDIR,10,bbox,projection,file[:-4]+"_ch10.tif")
        download_image(chm_1x1_filter,CHM_1x1_OUTDIR,1,bbox,projection,file[:-4]+"_ch1.tif")

def main():
  clipping("2017")

if __name__ == '__main__':
  main()

Total files: 8215


  0%|          | 0/8215 [00:00<?, ?it/s]

one more print below
I am working


  0%|          | 0/8215 [00:03<?, ?it/s]


SystemExit: Check if the files land where they are supposed to

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


### Faster variant
The code ran too slow so I addded Concurrency and batch requests. This massively speeds up the progression


In [57]:
"""
This code is responsible for "clipping" Dynamic world, Canopy Height models so that for each TIF file from NASA, there is a corresponding CHM models for comparison and Dynamic World to filter out everything that is not a tree
"""

# Imports
import rasterio
import os
import ee
import sys
from tqdm import tqdm
from datetime import datetime, timedelta
from pyproj import Transformer
import requests
import shutil
import concurrent.futures

# Directory in Google drive <-- Modify so that it points to your datasets
DATASET_PATH = "/content/drive/MyDrive/Summer internship - Research/Canopy_height_summer2024/Dataset/"

# Earth Engine setup <-- MODIFY
ee.Authenticate()
ee.Initialize(project="ee-rakinali00")  # Change this

# Global Variables
CHM_10X10 = ee.Image("users/nlang/ETH_GlobalCanopyHeight_2020_10m_v1")
dynamic_world = ee.ImageCollection("GOOGLE/DYNAMICWORLD/V1")
CHM_1x1 = ee.ImageCollection("projects/meta-forest-monitoring-okw37/assets/CanopyHeight")

# Check if paths exists. Returns true or sys.exit()
def check_path(path):
    if not os.path.exists(path):
        print("Path: ", path)
        print("Path does not exist")
        sys.exit()
    else:
        return True

def make_dir(path):
    if not os.path.exists(path):
        os.makedirs(path) # Create directory if it doesn't exist


def get_dateRange(file):
    """
    Extracts the date range from the filename.
    """
    parts = os.path.basename(file).split("_")
    date_str = None
    for part in parts:
        if len(part) == 8 and part.isdigit():
            date_str = part
            break

    if date_str is None:
        print(f"File: {file}")
        print("Cannot parse date from file name")
        return None, None

    try:
        start_date = datetime.strptime(date_str, "%Y%m%d")
        end_date = start_date + timedelta(days=10)
    except ValueError as e:
        print(f"Error parsing date from file {file}: {e}")
        return None, None

    return ee.Date(start_date.strftime("%Y-%m-%d")), ee.Date(end_date.strftime("%Y-%m-%d"))


def download_image(image, out_directory, scale, bbox, crs, file_name):
    url = image.getDownloadURL({'scale': scale, 'region': bbox, 'format': "GEO_TIFF", 'formatOptions': {'cloudOptimized': True}, 'crs': crs})
    r = requests.get(url, stream=True)
    if r.status_code != 200:
        r.raise_for_status()

    filename = os.path.join(out_directory, file_name)
    with open(filename, 'wb') as out_file:
        shutil.copyfileobj(r.raw, out_file)

def get_bbox(raster_file_path, target_crs="EPSG:4326"):
    """
    Extracts the bounding box from a raster file, transforms it to the specified CRS,
    and returns the bounding box as an Earth Engine Geometry Polygon.
    """
    with rasterio.open(raster_file_path) as src:
        xmin, ymin, xmax, ymax = src.bounds
        transformer = Transformer.from_crs(src.crs, target_crs, always_xy=True)
        x1, y1 = transformer.transform(xmin, ymax)
        x2, y2 = transformer.transform(xmax, ymin)
        projection = src.crs.to_string()

    bbox = ee.Geometry.Polygon([[x1, y1], [x1, y2], [x2, y2], [x2, y1], [x1, y1]])
    return bbox, projection

def process_file(file_path, year):
    bbox, projection = get_bbox(file_path)
    start_date, end_date = get_dateRange(file_path)

    dynamic_world_filter = (dynamic_world.filterDate(start_date, end_date).filterBounds(bbox)).median().clip(bbox)
    chm_1x1_filter = CHM_1x1.filterBounds(bbox).first()

    # Output directories
    CHM_10X10_OUTDIR = os.path.join(DATASET_PATH, year + "/CHM_10X10")
    CHM_1x1_OUTDIR = os.path.join(DATASET_PATH, year + "/CHM_1x1/")
    dynamic_world_OUTDIR = os.path.join(DATASET_PATH, year + "/Dynamic_World/")
    make_dir(CHM_10X10_OUTDIR)
    make_dir(CHM_1x1_OUTDIR)
    make_dir(dynamic_world_OUTDIR)

    if len(dynamic_world_filter.bandNames().getInfo()) > 0:
        file_name_dw = os.path.basename(file_path)[:-4] + "_dw.tif"
        download_image(dynamic_world_filter, dynamic_world_OUTDIR, 10, bbox, projection, file_name_dw)
    else:
        print("File", file_path, "has been skipped")
        return

    download_image(CHM_10X10, CHM_10X10_OUTDIR, 10, bbox, projection, os.path.basename(file_path)[:-4] + "_ch10.tif")
    download_image(chm_1x1_filter, CHM_1x1_OUTDIR, 1, bbox, projection, os.path.basename(file_path)[:-4] + "_ch1.tif")

def clipping(year):
    """
    Process each TIF file for the specified year.
    """
    tif_files_path = os.path.join(DATASET_PATH, year + "/NASA TIF Files/")
    if check_path(tif_files_path):
        tif_files = [os.path.join(tif_files_path, file) for file in os.listdir(tif_files_path) if file.endswith('.tif')]
        if not tif_files:
            sys.exit("There are no files in NASA TIF Files")
        print("Total files:", len(tif_files))

    with concurrent.futures.ThreadPoolExecutor() as executor:
        list(tqdm(executor.map(lambda file: process_file(file, year), tif_files), total=len(tif_files)))

def main():
    clipping("2017")

if __name__ == '__main__':
    main()


Total files: 8215


  1%|          | 45/8215 [00:58<2:57:03,  1.30s/it]


KeyboardInterrupt: 

## Masking out pixels
This part of the code is reponsible for the masking of building, houses etcetera. It should only store trees
